<a href="https://colab.research.google.com/github/FabioSicca/Organizacion-de-datos/blob/main/Spark2_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

12) Obtenga la matriz de distancias euclídeas para todos los contenidos que están en Marte. ¿Cuáles son los dos contenidos que están a menor distancia? (⭐⭐)

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=4725133a12ed7d815be8d13f8074a03ed01dcaa26c6f35ccc202fb39b5292380
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
type(sc)

pyspark.context.SparkContext

In [ ]:
geo_tags = spark.read.csv("/content/drive/MyDrive/Wikipedia dataset/geo_tags.csv", header=True, inferSchema=True, multiLine=True, escape='"')
contents = spark.read.csv("/content/drive/MyDrive/Wikipedia dataset/contents.csv", header=True, inferSchema=True, multiLine=True, escape='"')

In [ ]:
rdd_geo_tags = geo_tags.rdd
rdd_contents = contents.rdd

Filtro los geo tags a Marte y acomodo el rdd para que quede con clave, valor para poder hacer el join con contents


In [ ]:
mars_geo_tags = rdd_geo_tags.filter(lambda x: x.gt_globe == "mars").map(lambda x: (x.gt_page_id, (x.gt_lat,x.gt_lon)))

Uno los rdds de contenidos y de geo tags de Marte en base al page_id (al hacer un "inner join" también se filtran los datos).
Cacheo porque veo que es un rdd finito (1437 registros), para optimizar

In [ ]:
mars_content = rdd_contents.map(lambda x: (x.id, x.title)).join(mars_geo_tags).map(lambda x: (x[1][0], x[1][1]))
mars_content.cache()

#El formato de mars_content quedaría como (page_title, (lat, long))

PythonRDD[37] at RDD at PythonRDD.scala:53

Realizo todas las posibles combinaciones entre las pags, elimino las combinaciones de un registro consigo mismo y calculo la distancia


In [ ]:
#Filtro aquellas paginas que si bien no son la misma, hablan de lo mismo y por lo tanto tienen la misma ubicacion o distancia = 0
total_combinations = mars_content.cartesian(mars_content).filter(lambda x: x[0][1] != x[1][1]) #registro=((pg1,(lat1,lon1)), (pg2,(lat2,lon2)))
total_combinations.take(5)

[(('Centauri Montes', (-38.67, 95.52)), ('Luga (cráter)', (-44.25, 312.58))),
 (('Centauri Montes', (-38.67, 95.52)),
  ('South (cráter marciano)', (-76.94, 21.91))),
 (('Centauri Montes', (-38.67, 95.52)), ('Utopia Planitia', (46.7, 117.5))),
 (('Centauri Montes', (-38.67, 95.52)),
  ('Rutherford (cráter marciano)', (19.03, 349.41))),
 (('Centauri Montes', (-38.67, 95.52)), ('Beruri (cráter)', (5.2, 81.16)))]

In [ ]:
closest_contents_on_mars = total_combinations.map(lambda x: ( (x[0],x[1]) , (((x[0][1][0]-x[1][1][0])**2 + (x[0][1][1]-x[1][1][1])**2) **(1/2) ))).reduce(lambda r1,r2: r1 if (r1[1]<r2[1]) else r2)
                                                           #clave=(pg1,pg2)                           valor=(distancia)

print(closest_contents_on_mars)

((('Mackinac Island (meteorito)', (-2.11722222, 354.47710278)), ('Block Island (meteorito)', (-2.11690278, 354.48254167))), 0.005448262690585825)


Revisando en Wikipedia, efectivamente veo que las ubicaciones de ambos meteoritos (Mackinac Island y Block Island) tienen coordenadas muy cercanas y acá se observa que su distancia es 0.005448262690585825

#Código en limpio:

In [ ]:
rdd_geo_tags = geo_tags.rdd
rdd_contents = contents.rdd

mars_geo_tags = rdd_geo_tags.filter(lambda x: x.gt_globe == "mars").map(lambda x: (x.gt_page_id, (x.gt_lat,x.gt_lon)))

mars_content = rdd_contents.map(lambda x: (x.id, x.title)).join(mars_geo_tags).map(lambda x: (x[1][0], x[1][1]))
mars_content.cache()
#El formato de mars_content quedaría como (page_title, (lat, long))

#Filtro aquellas paginas que si bien no son la misma, hablan de lo mismo y por lo tanto tienen la misma ubicacion o distancia = 0
total_combinations = mars_content.cartesian(mars_content).filter(lambda x: x[0][1] != x[1][1]) #registro=((pg1,(lat1,lon1)), (pg2,(lat2,lon2)))

closest_contents_on_mars = total_combinations.map(lambda x: ( (x[0],x[1]) , (((x[0][1][0]-x[1][1][0])**2 + (x[0][1][1]-x[1][1][1])**2) **(1/2) ))).reduce(lambda r1,r2: r1 if (r1[1]<r2[1]) else r2)
                                                           #clave=(pg1,pg2)                           valor=(distancia)
print(closest_contents_on_mars)

((('Mackinac Island (meteorito)', (-2.11722222, 354.47710278)), ('Block Island (meteorito)', (-2.11690278, 354.48254167))), 0.005448262690585825)
